In [1]:
import pandas as pd

In [16]:
df = pd.read_csv('../../Leagues/1st_2nd_tiers_top_5_leagues.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.sort_values(by=['year', 'country', 'tier', 'position'], ascending=[True, True, True, True], inplace=True)

In [17]:
df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
441,2014,ES,LaLiga,1,FC Barcelona,1,31,25.9,17,19480000,603800000,NaN,NaN,False
440,2014,ES,LaLiga,1,Real Madrid,2,41,24.7,19,19210000,787800000,NaN,NaN,True
442,2014,ES,LaLiga,1,Atlético de Madrid,3,32,26.2,16,11130000,356000000,NaN,NaN,True
443,2014,ES,LaLiga,1,Valencia CF,4,26,24.5,13,8850000,230000000,NaN,NaN,False
444,2014,ES,LaLiga,1,Sevilla FC,5,34,25.5,14,4900000,166750000,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,2023,L,2-Bundesliga,2,1.FC Magdeburg,14,33,26.3,16,547000,18050000,NaN,NaN,False
1216,2023,L,2-Bundesliga,2,Eintracht Braunschweig,15,35,26.7,14,504000,17630000,NaN,NaN,False
1217,2023,L,2-Bundesliga,2,SV Wehen Wiesbaden,16,31,26.1,15,541000,16780000,NaN,NaN,False
1218,2023,L,2-Bundesliga,2,FC Hansa Rostock,17,35,26.2,14,419000,14650000,NaN,NaN,False


In [19]:
# Crear una copia del DataFrame para trabajar con seguridad
df_copy = df.copy()

# Ordenar por año, liga y posición para facilitar las comparaciones
df_copy.sort_values(by=['year', 'league', 'position'], inplace=True)

# Imputar valores en las columnas 'has_relegated' y 'has_promoted'
for idx, row in df_copy.iterrows():
    year, league, team, position = row['year'], row['league'], row['team'], row['position']
    
    # Datos del siguiente año
    next_year_teams = df_copy[(df_copy['year'] == year + 1) & (df_copy['team'] == team)]
    
    # Datos del año anterior
    prev_year_teams = df_copy[(df_copy['year'] == year - 1) & (df_copy['team'] == team)]
    
    # Ascenso: Primera posición en segunda división y aparece en la liga superior
    if league.endswith('2') and position <= 2:  # Asumimos 2 primeros ascienden directamente
        if not next_year_teams.empty and next_year_teams.iloc[0]['league'] != league:
            df_copy.loc[idx, 'has_promoted'] = True
        else:
            df_copy.loc[idx, 'has_promoted'] = False

    # Descenso: Últimas posiciones en primera división
    elif not league.endswith('2') and position >= 18:  # Asumimos 3 últimos descienden directamente
        if not next_year_teams.empty and next_year_teams.iloc[0]['league'] != league:
            df_copy.loc[idx, 'has_relegated'] = True
        else:
            df_copy.loc[idx, 'has_relegated'] = False

    # Playoffs: Equipos en posiciones de playoffs
    if league.endswith('2') and position in [3, 4, 5, 6]:  # Ejemplo para playoffs (ajustar según liga)
        if not next_year_teams.empty and next_year_teams.iloc[0]['league'] != league:
            df_copy.loc[idx, 'has_promoted'] = True
        else:
            df_copy.loc[idx, 'has_promoted'] = False

    if not league.endswith('2') and position in [16, 17]:  # Ejemplo para playoffs (ajustar según liga)
        if not next_year_teams.empty and next_year_teams.iloc[0]['league'] != league:
            df_copy.loc[idx, 'has_relegated'] = True
        else:
            df_copy.loc[idx, 'has_relegated'] = False

# Los valores que no pudieron imputarse quedan nulos


C:\Users\Tore\AppData\Local\Temp\ipykernel_21252\4254105455.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, 'has_relegated'] = False
C:\Users\Tore\AppData\Local\Temp\ipykernel_21252\4254105455.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.loc[idx, 'has_promoted'] = True


In [23]:
df = df_copy

In [24]:
df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
1047,2014,L,2-Bundesliga,2,FC Ingolstadt 04,1,30,25.8,12,641000,19230000,NaN,NaN,False
1053,2014,L,2-Bundesliga,2,SV Darmstadt 98,2,28,26.3,3,459000,12850000,NaN,NaN,False
1049,2014,L,2-Bundesliga,2,Karlsruher SC,3,31,25.4,10,556000,17250000,NaN,NaN,False
1042,2014,L,2-Bundesliga,2,1.FC Kaiserslautern,4,30,24.3,11,835000,25050000,NaN,NaN,False
1040,2014,L,2-Bundesliga,2,RB Leipzig,5,35,24.8,15,933000,32650000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,2023,IT,Serie-B,2,Ternana Calcio,16,52,25.0,17,498000,25880000,False,NaN,False
1616,2023,IT,Serie-B,2,SSC Bari,17,45,26.9,14,556000,25000000,False,NaN,False
1622,2023,IT,Serie-B,2,Ascoli Calcio,18,48,26.3,18,402000,19310000,False,NaN,False
1625,2023,IT,Serie-B,2,Feralpisalò,19,43,26.5,10,281000,12100000,False,NaN,True


In [25]:
df.isna().sum()

year                   0
country                0
league                 0
tier                   0
team                   0
position               0
squad_depth            0
avg_age                0
foreigners             0
avg_market_value       0
market_value           0
has_relegated       1620
has_promoted        1905
has_won_titles         0
dtype: int64